<a href="https://colab.research.google.com/github/srikanth0904/multi-class-animal-classification-/blob/main/Copy_of_major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import files
uploaded =files.upload()

Saving Emp_Train_2.csv to Emp_Train_2.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


In [4]:
df=pd.read_csv("Emp_Train_2.csv")
df

,EmployeeID,Tenure,Salary,PerformanceScore,Department,JobRole,Layoff
0,1,7,105629,4,Engineering,Manager,0
1,2,15,82603,3,Finance,Developer,0
2,3,11,111974,4,Engineering,Executive,0
3,4,8,56399,1,HR,Manager,1
4,5,7,115382,4,Sales,Developer,0
...,...,...,...,...,...,...,...
995,996,1,102240,3,Sales,Executive,0
996,997,8,94315,4,Finance,Analyst,0
997,998,7,99249,3,Sales,Consultant,0
998,999,3,100587,1,Marketing,Developer,0


In [5]:
df.isnull().sum()

,0
EmployeeID,0
Tenure,0
Salary,0
PerformanceScore,0
Department,0
JobRole,0
Layoff,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   EmployeeID        1000 non-null   int64 
 1   Tenure            1000 non-null   int64 
 2   Salary            1000 non-null   int64 
 3   PerformanceScore  1000 non-null   int64 
 4   Department        1000 non-null   object
 5   JobRole           1000 non-null   object
 6   Layoff            1000 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 54.8+ KB


In [9]:
df['Layoff'].value_counts()

,count
Layoff,
0,804
1,196


In [11]:
''' Train Test Split'''
X=df.drop(['EmployeeID','Layoff'],axis=1) # Drop EmployeeID and the target variable Layoff
y=df['Layoff'] # Set Layoff as the target variable
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test=train_test_split(X,y, test_size=0.33,stratify=y,random_state=42)

In [12]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify categorical columns (assuming they are still 'object' dtype)
categorical_cols = X_train.select_dtypes(include='object').columns

# Create a column transformer for one-hot encoding
# It will apply OneHotEncoder to categorical columns and leave others unchanged
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough' # Keep other columns (numerical) as they are
)

# Apply the preprocessing to the training data
X_train_processed = preprocessor.fit_transform(X_train)

# Now you can apply SMOTE to the processed training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_processed, y_train)

display(X_train_res.shape)
display(y_train_res.value_counts())

(1078, 13)

,count
Layoff,
0,539
1,539


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Apply the same preprocessing to the test data
X_test_processed = preprocessor.transform(X_test)


rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(X_train_res, y_train_res)
y_pred_rf = rf.predict(X_test_processed) # Use the preprocessed test data for prediction
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.7696969696969697


In [19]:
print("Report:\n", classification_report(y_test, y_pred_rf))


Report:
               precision    recall  f1-score   support

           0       0.81      0.94      0.87       265
           1       0.24      0.08      0.12        65

    accuracy                           0.77       330
   macro avg       0.52      0.51      0.49       330
weighted avg       0.69      0.77      0.72       330

